In [2]:
import numpy as np
import nrrd

In [ ]:
# convert this bad boy to a normal script
def import_flow(
    paths: tuple[str, str, str],
    venc: float,
    slope: float,
    intercept: float,
    timestep: float,
) -> np.ndarray:
    """Import 4D flow phase data, convert to velocity, and return as a 5D numpy array.

    Args:
        paths (tuple[str, str, str]): _description_
        vencs (tuple[int, int, int]): _description_
        phase_range (int, optional): _description_. Defaults to 4096.
        check (None | int, optional): _description_. Defaults to None.

    Returns:
        np.ndarray: 5-dimensional array of flow data (vel component, x, y, z, t)

    """
    vels = []
    
    for i, path in enumerate(paths):
        # read in the nrrd file
        data, header = nrrd.read(f"{path} {timestep}.nrrd")
        data[data < -199] = 0  # set weird cropped values to 0
        data = data/100 # convert to m/s
        
        vels.append(data)
        
    return vels

In [3]:
phase_directions = ["RL", "AP", "FH"]
paths = [f"{direction}_4D/{903+i} MR DelRec - 4D PC_{direction} - 24 frames Volume Sequence by TriggerTime" for i, direction in enumerate(phase_directions)]


mask_TL = nrrd.read("Segmentation_TL_4DFLOWspace.nrrd")[0]
mask_FL = nrrd.read("Segmentation_FL_4DFLOWspace.nrrd")[0]

In [11]:
dx = np.array([1.18644, 1.18644, 2.5])/1000 # convert to meters



# convert this bad boy to a normal script
def import_flow(
    paths: tuple[str, str, str],
    venc: float,
    slope: float,
    intercept: float,
    timestep: float,
) -> np.ndarray:
    """Import 4D flow phase data, convert to velocity, and return as a 5D numpy array.

    Args:
        paths (tuple[str, str, str]): _description_
        vencs (tuple[int, int, int]): _description_
        phase_range (int, optional): _description_. Defaults to 4096.
        check (None | int, optional): _description_. Defaults to None.

    Returns:
        np.ndarray: 5-dimensional array of flow data (vel component, x, y, z, t)

    """
    vels = []
    
    for i, path in enumerate(paths):
        # read in the nrrd file
        data, header = nrrd.read(f"{path} {timestep}.nrrd")
        data[data < -199] = 0  # set weird cropped values to 0
        data = data/100 # convert to m/s
        
        vels.append(data)
        
    return vels, header

vel_tl_full = np.zeros((3, 288, 288, 40, 24)) # hardcoded for now
for nt in range(24):
    vel_t, header = import_flow(paths, 200, 0.09768009768009, -200.0, nt)
    vel_tl = []
    vel_fl = []
    for i, vel in enumerate(vel_t):
        
        vel_tl.append(vel * mask_TL)
        vel_fl.append(vel * mask_FL)
        
    vel_tl_full[0, :, :, :, nt] = vel_tl[0]
    vel_tl_full[1, :, :, :, nt] = vel_tl[1]
    vel_tl_full[2, :, :, :, nt] = vel_tl[2]

    #imageToVTK(f"UM16_TL_{nt}", cellData={"velocity": (vel_tl[1], -vel_tl[2], -vel_tl[0])}, spacing=dx.tolist())
    #imageToVTK(f"UM16_FL_{nt}", cellData={"velocity": (vel_fl[1], -vel_fl[2], -vel_fl[0])}, spacing=dx.tolist())
    print(f"{nt} completed!")
    
header["dimension"] = 5
header["sizes"] = [3, 288, 288, 40, 24]
header["kinds"] = ["vector", "domain", "domain", "domain", "list"]
old_direction = header["space directions"].tolist()
header["space directions"] = [None] + old_direction + [None]
print(header["space directions"])
#nrrd.write("UM16_TL_4DFLOW_TEST.nrrd", vel_tl_full, header)

0 completed!
1 completed!
2 completed!
3 completed!
4 completed!
5 completed!
6 completed!
7 completed!
8 completed!
9 completed!
10 completed!
11 completed!
12 completed!
13 completed!
14 completed!
15 completed!
16 completed!
17 completed!
18 completed!
19 completed!
20 completed!
21 completed!
22 completed!
23 completed!
[None, [0.6176123075716905, 1.0102566672806637, -0.07468636559087288], [-0.13977715082611109, -0.0016487629766575264, -1.1781770576368467], [-2.1141470884664617, 1.3108729827097434, 0.24898538834010298], None]


In [8]:
import numpy as np
import nrrd

# Define data shape
vector_components = 3
x, y, z = 64, 64, 32
timepoints = 5

# Create synthetic data: shape = (3, x, y, z, t)
data = np.random.rand(3, x, y, z).astype(np.float32)
print(data.shape)

# Define header metadata
header = {
    'type': 'float',
    'dimension': 3,
    'sizes': data.shape,
    'kinds': ['vector', 'domain', 'domain', 'domain'],
    'endian': 'little',
    'encoding': 'raw',
    'space': 'right-anterior-superior',
    #'space directions': [[None, None, None], [2,1,0], [0,0.5,0], [0,0,1]],
    'space origin': [0.0, 0.0, 0.0]
}

# Save NRRD file
nrrd.write('vector_field.nrrd', data, header)
print("NRRD file saved as vector_field.nrrd")

(3, 64, 64, 32)
NRRD file saved as vector_field.nrrd


In [18]:
64*64

4096